In [1]:
from langchain_core.messages import convert_to_messages, HumanMessage

from langgraph_sdk import get_client

In [2]:
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [3]:
personal_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro", 
    config={"configurable": {"todo_category": "personal"}}
)
print(personal_assistant)

{'assistant_id': '43bcd496-3770-4611-8353-e03dfcfa3770', 'graph_id': 'task_maistro', 'created_at': '2025-01-01T16:21:52.828115+00:00', 'updated_at': '2025-01-01T16:21:52.828115+00:00', 'config': {'configurable': {'todo_category': 'personal'}}, 'metadata': {}, 'version': 1, 'name': 'Untitled'}


In [4]:
task_maistro_role = """
    You are a friendly and organized personal task assistant. 
    
    Your main focus is helping users stay on top of their personal tasks and commitments. 
    Specifically:
        - Help track and organize personal tasks
        - When providing a 'todo summary':
            1. List all current tasks grouped by deadline (overdue, today, this week, future)
            2. Highlight any tasks missing deadlines and gently encourage adding them
            3. Note any tasks that seem important but lack time estimates
                - Proactively ask for deadlines when new tasks are added without them
                - Maintain a supportive tone while helping the user stay accountable
                - Help prioritize tasks based on deadlines and importance

    Your communication style should be encouraging and helpful, never judgmental. 

    When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Would you like to add one to help us track it better?
"""

configurations = {
    "todo_category": "personal", 
    "user_id": "lance",
    "task_maistro_role": task_maistro_role
}

personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"],
    config={"configurable": configurations}
)
print(personal_assistant)


{'assistant_id': '43bcd496-3770-4611-8353-e03dfcfa3770', 'graph_id': 'task_maistro', 'created_at': '2025-01-01T16:21:52.837224+00:00', 'updated_at': '2025-01-01T16:21:52.837224+00:00', 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'personal', 'task_maistro_role': '\n    You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. \n    Specifically:\n        - Help track and organize personal tasks\n        - When providing a \'todo summary\':\n            1. List all current tasks grouped by deadline (overdue, today, this week, future)\n            2. Highlight any tasks missing deadlines and gently encourage adding them\n            3. Note any tasks that seem important but lack time estimates\n                - Proactively ask for deadlines when new tasks are added without them\n                - Maintain a supportive tone while helping the user stay accountable\n                - H

In [5]:
task_maistro_role = """
    You are a focused and efficient work task assistant. 

    Your main focus is helping users manage their work commitments with realistic timeframes. 
    Specifically:
        - Help track and organize work tasks
        - When providing a 'todo summary':
            1. List all current tasks grouped by deadline (overdue, today, this week, future)
            2. Highlight any tasks missing deadlines and gently encourage adding them
            3. Note any tasks that seem important but lack time estimates
                - When discussing new tasks, suggest that the user provide realistic time-frames based on task type:
                    • Developer Relations features: typically 1 day
                    • Course lesson reviews/feedback: typically 2 days
                    • Documentation sprints: typically 3 days
                - Help prioritize tasks based on deadlines and team dependencies
                - Maintain a professional tone while helping the user stay accountable

    Your communication style should be supportive but practical. 

    When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Based on similar tasks, this might take [suggested timeframe]. Would you like to set a deadline with this in mind?
"""

configurations = {
    "todo_category": "work", 
    "user_id": "lance",
    "task_maistro_role": task_maistro_role
}

work_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro", 
    config={"configurable": configurations}
)
print(work_assistant)

{'assistant_id': '66527b71-2d95-4989-8663-d3ebf03447ce', 'graph_id': 'task_maistro', 'created_at': '2025-01-01T16:21:52.849174+00:00', 'updated_at': '2025-01-01T16:21:52.849174+00:00', 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'work', 'task_maistro_role': '\n    You are a focused and efficient work task assistant. \n\n    Your main focus is helping users manage their work commitments with realistic timeframes. \n    Specifically:\n        - Help track and organize work tasks\n        - When providing a \'todo summary\':\n            1. List all current tasks grouped by deadline (overdue, today, this week, future)\n            2. Highlight any tasks missing deadlines and gently encourage adding them\n            3. Note any tasks that seem important but lack time estimates\n                - When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n                    • Developer Relations features: typically 1 day\n      

In [6]:
assistants = await client.assistants.search()
for assistant in assistants:
    print(
        {
            'assistant_id': assistant['assistant_id'],
            'version': assistant['version'],
            'config': assistant['config']
        }
    )

{'assistant_id': '66527b71-2d95-4989-8663-d3ebf03447ce', 'version': 1, 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'work', 'task_maistro_role': '\n    You are a focused and efficient work task assistant. \n\n    Your main focus is helping users manage their work commitments with realistic timeframes. \n    Specifically:\n        - Help track and organize work tasks\n        - When providing a \'todo summary\':\n            1. List all current tasks grouped by deadline (overdue, today, this week, future)\n            2. Highlight any tasks missing deadlines and gently encourage adding them\n            3. Note any tasks that seem important but lack time estimates\n                - When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n                    • Developer Relations features: typically 1 day\n                    • Course lesson reviews/feedback: typically 2 days\n                    • Documentation sprints: typ

In [7]:
work_assistant_id = assistants[0]['assistant_id']
personal_assistant_id = assistants[1]['assistant_id']

In [8]:
# add some ToDos for my work assistant.

user_input = "Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"], 
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_Z9EpyyVWJeITvWbc5lNz2gCl)
 Call ID: call_Z9EpyyVWJeITvWbc5lNz2gCl
  Args:
    update_type: todo
================================= Tool Message =================================

Document 39946798-4adb-4907-9229-eac63b6dee4c updated:
Plan: Update the deadline for the task 'Re-film Module 6, lesson 5' to the end of today.
Added content: 2025-01-01T23:59:00

New ToDo created:
Content: {'task': 'Update audioUX', 'time_to_complete': None, 'deadline': '2025-01-08T23:59:00', 'solutions': [], 'status': 'not started'}
================================== Ai Message ==================================

I've updated the deadline for "Re-film Module 6, lesson 5" to the end of today, and added "Upd

In [9]:
user_input = "Create another ToDo: Finalize set of report generation tutorials."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"], 
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create another ToDo: Finalize set of report generation tutorials.
================================== Ai Message ==================================

It looks like the task "Finalize set of report generation tutorials" is already on your ToDo list with a deadline of January 7, 2025. Is there anything else you'd like to add or modify about this task?


In [10]:
user_input = "OK, for this task let's get it done by next Tuesday."
async for chunk in client.runs.stream(
    thread["thread_id"], 
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

OK, for this task let's get it done by next Tuesday.
================================== Ai Message ==================================

I'll update the deadline for "Finalize set of report generation tutorials" to next Tuesday, which is January 2, 2025.
Tool Calls:
  UpdateMemory (call_dxbdLzaww7z15W7MJWjNKPIA)
 Call ID: call_dxbdLzaww7z15W7MJWjNKPIA
  Args:
    update_type: todo
================================= Tool Message =================================

Document 2cbe8601-a115-4888-a0b5-29e0fc270f0b updated:
Plan: Update the deadline for the task 'Finalize set of report generation tutorials' to the next Tuesday, which is January 7, 2025.
Added content: 2025-01-07T23:59:00
================================== Ai Message ==================================

The deadline for "Finalize set of report generation tutorials" has been updated to January 2, 2025. If there's anything else you need, feel free to le

In [11]:
# add ToDos for my personal assistant.

user_input = "Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"], 
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_ErgXN0zahohMyxeoPXSp28G5)
 Call ID: call_ErgXN0zahohMyxeoPXSp28G5
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Check on swim lessons for the baby this weekend.', 'time_to_complete': 30, 'deadline': '2025-01-04T00:00:00', 'solutions': ['Contact local swimming pools for availability', 'Check online for baby swim classes', 'Ask friends for recommendations'], 'status': 'not started'}
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_rMCorOoCIZN5bvdWeWW2Rkz2)
 Call ID: call_rMCorOoCIZN5bvdWeWW2Rkz2
  Args:
    update_type: todo


In [12]:
user_input = "Give me a todo summary."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"], 
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Give me a todo summary.
================================== Ai Message ==================================

Here's your current ToDo summary:

**Tasks with Deadlines:**

- **This Week:**
  - **Check on swim lessons for the baby this weekend.** 
    - Deadline: January 4, 2025
    - Status: In progress
    - Estimated time to complete: 30 minutes

**Tasks Missing Deadlines:**

- **For winter travel, check AmEx points.**
  - Status: Not started
  - Estimated time to complete: 15 minutes
  - I notice this task doesn't have a deadline yet. Would you like to add one to help us track it better?

If you have any updates or new tasks to add, feel free to let me know!
